In [1]:
from squad_solver import *
from utils_squads import *

squad_solver = SquadSolver()
ampl = squad_solver.get_ampl()
ampl.eval('option presolve 0;')

params = {
    "budget": 1000000,
    "number_of_defenders": 4,
    "number_of_midfielders": 3,
    "number_of_forwards": 3,
    "min_chemistry": 15,
    "M":11
}

limit_rows = None # Number of rows to read from the csv file. PON 300 AQUÍ PORQUE SI NO TE VA A ERROR POR LA LICENCIA DE AMPL
data = squad_solver.read_csv_data("players_1867.csv", limit_rows=limit_rows)
ampl.export_data("players_"+str(limit_rows)+".dat")
summary_squad(data)
squad_solver.set_ampl_data(params)

# General components of all formulations
general_components = "Avg_Global, PositionGK, PositionDEF, PositionMID, PositionFOR, Budget, R6"
general_components += ", y, natCount, chemistryNat1, chemistryNat2, chemistryNat3"
formulation_1 = "problem Formulation_1: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R6;"
squad_solver.add_formulation("Formulation_1", formulation_1)
formulation_2 = "problem Formulation_2: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R34A, R34B, R34C, R6;"
squad_solver.add_formulation("Formulation_2", formulation_2)


########################################################
################# SUMMARY OF THE SQUAD #################
Number of players: 1864, number of regular players: 1679, number of icons: 107, number of heroes: 78.
The mean Overall rating of each player is 82.28809012875537
The mean price of each player is 26651.046137339057
########################################################



In [2]:
results_formulation_1 = squad_solver.solve("Formulation_1", print_results=True, print_summary=True)
squad_formulation_1 = squad_solver.get_squad()

Solving formulation: Formulation_1.
Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
8401 constraints, all linear; 177799 nonzeros
	4 equality constraints
	8397 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 0
  cut:cuts = 0
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  0
InfUnbdInfo  1
Presolve  0

Optimize a model with 8401 rows, 8580 columns and 177799 nonzeros
Model fingerprint: 0xb27e8831
Variable types: 0 continuous, 8580 integer (0 binary)
Coeffi

In [3]:
results_formulation_2 = squad_solver.solve("Formulation_2", print_results=True, print_summary=True)
squad_formulation_2 = squad_solver.get_squad()

Solving formulation: Formulation_2.
Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 0
  cut:cuts = 0
  mip:bestbound = 1
Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 0
  cut:cuts = 0
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  0
InfUnbdInfo

In [4]:
pd.DataFrame([results_formulation_1, results_formulation_2], index=["Formulation 1", "Formulation 2"]).drop("output", axis=1)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation 1,solved,8580,8401,0.0,23,1,0.484,91.363636
Formulation 2,solved,8580,13438,0.0,27,1,0.344,91.363636


### Try another value of M

In [5]:
params["M"] = len(data)
squad_solver.set_ampl_data(params)

In [6]:
results_formulation_1 = squad_solver.solve("Formulation_1", print_results=False, print_summary=True)
squad_formulation_1 = squad_solver.get_squad()

Solving formulation: Formulation_1.
Solver result:  solved
Number of Variables: 8580
Number of Constraints: 8401
Gap: 0.0
Number of Simplex Iterations: 22
Number of Branching Nodes: 1
                           name         player_type generic_position position  \
0                Nadine Angerer                Icon               GK       GK   
1                 Franco Baresi                Icon              DEF       CB   
2                   Bobby Moore                Icon              DEF       CB   
3      María Pilar León Cebrián              Normal              DEF       CB   
4   Alexsandro de Souza Ribeiro          Fantasy FC              DEF       CB   
5    Rodrigo Hernández Cascante              Normal              MID      CDM   
6          Aitana Bonmatí Conca              Normal              MID       CM   
7               Zinedine Zidane                Icon              MID      CAM   
8                  Birgit Prinz                Icon              FOR       ST   
9     

In [7]:
results_formulation_2 = squad_solver.solve("Formulation_2", print_results=False, print_summary=True)
squad_formulation_2 = squad_solver.get_squad()

Solving formulation: Formulation_2.
Solver result:  solved
Number of Variables: 8580
Number of Constraints: 13438
Gap: 0.0
Number of Simplex Iterations: 25
Number of Branching Nodes: 1
                           name         player_type generic_position position  \
0                Nadine Angerer                Icon               GK       GK   
1                 Franco Baresi                Icon              DEF       CB   
2                   Bobby Moore                Icon              DEF       CB   
3      María Pilar León Cebrián              Normal              DEF       CB   
4   Alexsandro de Souza Ribeiro          Fantasy FC              DEF       CB   
5    Rodrigo Hernández Cascante              Normal              MID      CDM   
6          Aitana Bonmatí Conca              Normal              MID       CM   
7               Zinedine Zidane                Icon              MID      CAM   
8                  Birgit Prinz                Icon              FOR       ST   
9    

In [8]:
pd.DataFrame([results_formulation_1, results_formulation_2], index=["Formulation 1", "Formulation 2"]).drop("output", axis=1)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation 1,solved,8580,8401,0.0,22,1,0.297,91.363636
Formulation 2,solved,8580,13438,0.0,25,1,0.344,91.363636
